In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [2]:
# Read stroke csv file
stroke_read = pd.read_csv('resources/stroke_data.csv')

stroke_read.head()


,sex,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1.0,63.0,0,1,1,4,1,228.69,36.6,1,1
1,1.0,42.0,0,1,1,4,0,105.92,32.5,0,1
2,0.0,61.0,0,0,1,4,1,171.23,34.4,1,1
3,1.0,41.0,1,0,1,3,0,174.12,24.0,0,1
4,1.0,85.0,0,0,1,4,1,186.21,29.0,1,1


In [3]:
stroke_read.count()

sex                  40907
age                  40910
hypertension         40910
heart_disease        40910
ever_married         40910
work_type            40910
Residence_type       40910
avg_glucose_level    40910
bmi                  40910
smoking_status       40910
stroke               40910
dtype: int64

In [4]:
stroke_read.nunique()

sex                     2
age                   111
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    2903
bmi                   370
smoking_status          2
stroke                  2
dtype: int64

In [5]:
# Checked for missing values then,
# Determine unique values
for x in stroke_read.columns:
    print(x, len(stroke_read[x].unique()))

sex 3
age 111
hypertension 2
heart_disease 2
ever_married 2
work_type 5
Residence_type 2
avg_glucose_level 2903
bmi 370
smoking_status 2
stroke 2


In [6]:
stroke_read['age'].value_counts().sort_index()

age
-9.0       1
-6.0       2
-5.0       6
-4.0       7
-3.0      11
          ..
 99.0     31
 100.0    14
 101.0     6
 102.0     7
 103.0     1
Name: count, Length: 111, dtype: int64

In [7]:
stroke_read = stroke_read[stroke_read['age'] >= 0]
stroke_read

,sex,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1.0,63.0,0,1,1,4,1,228.69,36.6,1,1
1,1.0,42.0,0,1,1,4,0,105.92,32.5,0,1
2,0.0,61.0,0,0,1,4,1,171.23,34.4,1,1
3,1.0,41.0,1,0,1,3,0,174.12,24.0,0,1
4,1.0,85.0,0,0,1,4,1,186.21,29.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
40905,1.0,38.0,0,0,0,4,1,120.94,29.7,1,0
40906,0.0,53.0,0,0,1,4,0,77.66,40.8,0,0
40907,1.0,32.0,0,0,1,2,0,231.95,33.2,0,0
40908,1.0,42.0,0,0,1,3,0,216.38,34.5,0,0


In [8]:
stroke_read['age'].value_counts().sort_index()

age
0.0      23
1.0      32
2.0      32
3.0      49
4.0      65
         ..
99.0     31
100.0    14
101.0     6
102.0     7
103.0     1
Name: count, Length: 104, dtype: int64

In [9]:
stroke_read["age_category"] = np.where(stroke_read['age']<=44, "18-44",
                           np.where(stroke_read['age']<=59, "45-59",
                           np.where(stroke_read['age']<=69, "60-69", "70+")))
del stroke_read['age']
stroke_read

,sex,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_category
0,1.0,0,1,1,4,1,228.69,36.6,1,1,60-69
1,1.0,0,1,1,4,0,105.92,32.5,0,1,18-44
2,0.0,0,0,1,4,1,171.23,34.4,1,1,60-69
3,1.0,1,0,1,3,0,174.12,24.0,0,1,18-44
4,1.0,0,0,1,4,1,186.21,29.0,1,1,70+
...,...,...,...,...,...,...,...,...,...,...,...
40905,1.0,0,0,0,4,1,120.94,29.7,1,0,18-44
40906,0.0,0,0,1,4,0,77.66,40.8,0,0,45-59
40907,1.0,0,0,1,2,0,231.95,33.2,0,0,18-44
40908,1.0,0,0,1,3,0,216.38,34.5,0,0,18-44


In [10]:
stroke_read.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40852 entries, 0 to 40909
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sex                40849 non-null  float64
 1   hypertension       40852 non-null  int64  
 2   heart_disease      40852 non-null  int64  
 3   ever_married       40852 non-null  int64  
 4   work_type          40852 non-null  int64  
 5   Residence_type     40852 non-null  int64  
 6   avg_glucose_level  40852 non-null  float64
 7   bmi                40852 non-null  float64
 8   smoking_status     40852 non-null  int64  
 9   stroke             40852 non-null  int64  
 10  age_category       40852 non-null  object 
dtypes: float64(3), int64(7), object(1)
memory usage: 3.7+ MB


In [11]:
stroke_read = stroke_read.dropna()
stroke_read.isnull().sum()

sex                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
age_category         0
dtype: int64

In [12]:
dummies_stroke_df = pd.get_dummies(stroke_read, dtype=int)
dummies_stroke_df

,sex,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_category_18-44,age_category_45-59,age_category_60-69,age_category_70+
0,1.0,0,1,1,4,1,228.69,36.6,1,1,0,0,1,0
1,1.0,0,1,1,4,0,105.92,32.5,0,1,1,0,0,0
2,0.0,0,0,1,4,1,171.23,34.4,1,1,0,0,1,0
3,1.0,1,0,1,3,0,174.12,24.0,0,1,1,0,0,0
4,1.0,0,0,1,4,1,186.21,29.0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40905,1.0,0,0,0,4,1,120.94,29.7,1,0,1,0,0,0
40906,0.0,0,0,1,4,0,77.66,40.8,0,0,0,1,0,0
40907,1.0,0,0,1,2,0,231.95,33.2,0,0,1,0,0,0
40908,1.0,0,0,1,3,0,216.38,34.5,0,0,1,0,0,0


In [13]:
# Split the data
y = dummies_stroke_df["stroke"]
X = dummies_stroke_df.drop("stroke", axis=1)

In [14]:
# Splitting the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [15]:
# Train Model
clf = LogisticRegression(random_state=1,max_iter=1000)
clf.fit(X_train, y_train)
# Make predictions
predictions = clf.predict(X_test)

print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

0.6863801037892882
              precision    recall  f1-score   support

           0       0.66      0.74      0.70      5066
           1       0.71      0.63      0.67      5147

    accuracy                           0.69     10213
   macro avg       0.69      0.69      0.69     10213
weighted avg       0.69      0.69      0.69     10213

[[3771 1295]
 [1908 3239]]


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network
model = Sequential()
model.add(Dense(60, input_shape=(X_train.shape[1],), activation="relu"))
model.add(BatchNormalization())
model.add(Dense(120, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Make predictions
predictions = model.predict(X_test_scaled)
predictions = (predictions > 0.5).astype(int)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, predictions))
print("Classification Report:")
print(classification_report(y_test, predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))


c:\Users\amino\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6833 - loss: 0.5861 - val_accuracy: 0.7296 - val_loss: 0.5234
Epoch 2/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7382 - loss: 0.5151 - val_accuracy: 0.7647 - val_loss: 0.4655
Epoch 3/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7725 - loss: 0.4650 - val_accuracy: 0.8074 - val_loss: 0.4090
Epoch 4/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7944 - loss: 0.4201 - val_accuracy: 0.8280 - val_loss: 0.3691
Epoch 5/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8147 - loss: 0.3850 - val_accuracy: 0.8474 - val_loss: 0.3245
Epoch 6/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8349 - loss: 0.3476 - val_accuracy: 0.8539 - val_loss: 0.3074
Epoch 7/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8499 - loss: 0.3233 - val_accuracy: 0.8636 - val_loss: 0.3031
Epoch 8/100
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8517 - loss: 0.3128 - val_accu

In [17]:
# Dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, r2_score

In [18]:
# Instantiate a Random Forest Classifier model
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=1)

In [19]:
# Fit the model with training data
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=1)

In [20]:
# Make predictions on the test set
predictions = rf_model.predict(X_test)

In [21]:
# Evaluate the model with Confusion Matrix and accuracy score
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

acc_score = accuracy_score(y_test, predictions)

In [22]:
# Generate a confusion matrix for the model
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,5006,60
Actual 1,0,5147


In [23]:
# Look at the accuracy score 
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9941251346323313


In [24]:
# Look at the classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5066
           1       0.99      1.00      0.99      5147

    accuracy                           0.99     10213
   macro avg       0.99      0.99      0.99     10213
weighted avg       0.99      0.99      0.99     10213



In [26]:
import pickle
with open('model.pkl','wb') as f:
  pickle.dump(rf_model,f)